In [1]:
# if colab

# !pip install pybullet
# !pip install gym
# !apt-get install python-opengl -y
# !apt install xvfb -y
# !pip install gym pyvirtualdisplay > /dev/null 2>&1
# !pip install -q git+https://github.com/tensorflow/examples.git

In [2]:
import os
import cv2
import tensorflow as tf 
from tensorflow.keras import layers, models
import numpy as np 
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
import pybullet_envs
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

In [3]:
np_seed = 654765645
tf_seed = 776644345
np.random.seed(np_seed)
tf.random.set_seed(tf_seed)

# check if GPU
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# colab

# from google.colab import drive
# drive.mount('/content/drive')

# root_dir = "drive/My Drive/"
# base_dir = root_dir + 'CPCtesting'
# os.makedirs(base_dir,exist_ok=True)

# train_dir = base_dir + '/train'
# os.makedirs(train_dir,exist_ok=True)

# model_dir = base_dir + '/model'
# os.makedirs(model_dir,exist_ok=True)

# if local machine
base_dir = "."

train_dir = base_dir + '/train'
os.makedirs(train_dir,exist_ok=True)

model_dir = base_dir + '/model'
os.makedirs(model_dir,exist_ok=True)

logs_base_dir = base_dir + '/logs'

log_dir = base_dir + '/training_logs_save'




# tensorboard
%load_ext tensorboard
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [5]:
# get data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [6]:
class CPCModel(tf.keras.Model):
    def __init__(self,code_size, predict_terms, terms=4, units=256, image_size=64, channels=3):
        super(CPCModel, self).__init__()
        self.code_size = code_size
        self.predict_terms = predict_terms
        self.terms = terms
        self.units = units
        self.image_size = image_size
        self.channels = channels

        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.lrelu1 = tf.keras.layers.LeakyReLU()
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.lrelu2 = tf.keras.layers.LeakyReLU()
        self.conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.lrelu3 = tf.keras.layers.LeakyReLU()
        self.conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn4 = tf.keras.layers.BatchNormalization()
        self.lrelu4 = tf.keras.layers.LeakyReLU()
        self.flatten = tf.keras.layers.Flatten()
        self.dense5 = tf.keras.layers.Dense(units=256, activation='linear')
        self.bn5 = tf.keras.layers.BatchNormalization()
        self.lrelu5 = tf.keras.layers.LeakyReLU()
        self.dense6 = tf.keras.layers.Dense(units=code_size, activation='linear', name='encoder_embedding')

        self.gru = tf.keras.layers.GRU(units, return_sequences=False, name='ar_context')
        self.linear = tf.keras.layers.Dense(predict_terms*code_size, activation='linear')    
   
    def encoding(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.lrelu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.lrelu2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.lrelu3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.lrelu4(x)
        x = self.flatten(x)
        x = self.dense5(x)
        x = self.bn5(x)
        x = self.lrelu5(x)
        z = self.dense6(x)
        return z
  
    def get_context(self, x):
        z = self.encoding(x)
        z = tf.reshape(z, [-1, self.terms, self.code_size])
        c = self.gru(z)
        return c
    def get_prediction(self, x):
        c = self.get_context(x)
        z_hats = self.linear(c)
        z_hat = tf.reshape(z_hats, [-1, self.predict_terms, self.code_size])
        return z_hat

    def optimizer(self):
        pass

    def loss(self,weights,biases,labels,inputs,num_samples,num_classes): 
        loss = tf.nn.nce_loss(
        weights, biases, labels, inputs, num_sampled, num_classes, num_true=1,
        sampled_values=None, remove_accidental_hits=False, name='nce_loss')
        return loss
  
    def call(self,inputs):
        x_tm, x_tp = inputs
        x_tm = tf.reshape(x_tm, [-1, self.image_size, self.image_size, self.channels])
        x_tp = tf.reshape(x_tp, [-1, self.image_size, self.image_size, self.channels])
        z_hat = self.get_prediction(x_tm)
        z_tp = self.encoding(x_tp)
        z_tp = tf.reshape(z_tp, [-1, self.predict_terms, self.code_size])
        dot_prods = tf.reduce_mean(tf.reduce_mean(z_hat*z_tp, axis=-1), axis=-1, keepdims=True)
        probs = tf.sigmoid(dot_prods)
        return probs


  # def save(self):
  #       f1 = os.path.join(folder,'target_actor')
  #       f2 = os.path.join(folder, 'target_critic')
  #       f3 = os.path.join(folder, 'actor')
  #       f4 = os.path.join(folder, 'critic')
  #       self.target_actor.save(f1)
  #       self.target_critic.save(f2)
  #       self.actor.save(f3)
  #       self.critic.save(f4)


  # def load(self):
  #   pass

In [7]:
class ReplayBuffer():
    def __init__(self,state_space,action_space,capacity,batch):
        self.capacity = capacity
        self.batch = batch
        
        self.avaliable_batch = 0
        self.idx = 0
        self.entries = 0 
        
        self.states = np.empty((self.capacity,state_space),dtype = np.float32)
        self.next_states = np.empty((self.capacity,state_space),dtype = np.float32)
        self.actions = np.empty((self.capacity,action_space),dtype = np.float32)
        self.rewards = np.empty((self.capacity,1),dtype = np.float32)
        self.not_dones = np.empty((self.capacity, 1), dtype=np.float32)
        
    def add(self,state,next_state,action,reward,done):
        np.copyto(self.states[self.idx], state)
        np.copyto(self.actions[self.idx], action)
        np.copyto(self.rewards[self.idx], reward)
        np.copyto(self.next_states[self.idx], next_state)
        np.copyto(self.not_dones[self.idx], not done)
        #self.avaliable_batch= (self.avaliable_batch + 1) if self.avaliable_batch < self.batch else self.batch
        #self.entries = (self.entries + 1) if self.entries < self.capacity else self.capacity
        self.idx = (self.idx + 1) % self.capacity
        
    def sample(self):
        #num = self.avaliable_batch
        #if(num > self.batch):
        #    num = self.batch
        #print('avaliable_batch: ',self.avaliable_batch, "entries: ", self.entries,'capacity: ', self.capacity)
        idx = np.random.choice(self.capacity,size = self.batch,replace=False)
        #print('test idx: ', idx)
        
        states = tf.convert_to_tensor(self.states[idx])
        next_states = tf.convert_to_tensor(self.next_states[idx])
        actions = tf.convert_to_tensor(self.actions[idx])
        rewards = tf.convert_to_tensor(self.rewards[idx])
        not_dones = tf.convert_to_tensor(self.not_dones[idx])
        
        return states,next_states,actions,rewards,not_dones
    
    def fill_buffer(self,timesteps,state,prev_timesteps):
        print('sim test: ',env._max_episode_steps,":",timesteps)
        for step in range(timesteps):
            action = env.action_space.sample()
            next_state, reward, done, info = env.step(action)
            np.copyto(self.states[step], state)
            np.copyto(self.actions[step], action)
            np.copyto(self.rewards[step], reward)
            np.copyto(self.next_states[step], next_state)
            np.copyto(self.not_dones[step], not done)
            state = next_state
            if(done):
                print("step: ", step)
                state = env.reset()
                print('done seeding replay buffer')            
            
        

class Actor(tf.keras.Model):
    def __init__(self,action_space,critic,actor_lr = 0.001,variance = 0.2):
        super(Actor,self).__init__()
        
        #params
        self.std = np.sqrt(variance)
        self.noise_flag = 1.0
        self.action_space = action_space
        
        #optimizer
        self.opt = tf.keras.optimizers.Adam(actor_lr)
        self.critic = critic
       
        #model
        self.dense1 = tf.keras.layers.Dense(400,activation = 'relu',dtype='float32')
        self.dense2 = tf.keras.layers.Dense(300,activation='relu',dtype='float32')
        self.dense3 = tf.keras.layers.Dense(action_space,activation = 'tanh',dtype='float32')    
        
    def loss(self,states,actions):
        actions = self(states)
        #stateactions = tf.concat([states,actions],-1)
        Q = self.critic(states,actions)
        loss = - tf.reduce_mean(Q)
        return loss
    
    def update(self,states,actions):
        with tf.GradientTape() as tape:
            loss = self.loss(states,actions)

        grad = tape.gradient(loss,self.trainable_variables)
        self.opt.apply_gradients(zip(grad, self.trainable_variables))
        #print('actor loss: ', loss ,"\n" )
        return loss
    
    def set_noise_flag(self,num):
        self.noise_flag = np.float32(not not num)
    
    def continous_noise(self):
        result = np.random.normal(0,self.std,size=(self.action_space,))
        return self.noise_flag *result
    
    def call(self,x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

class Critic(tf.keras.Model):
    def __init__(self,critic_lr = 0.001):
        super(Critic,self).__init__()
        
        # optimizer
        self.opt = tf.keras.optimizers.Adam(critic_lr)
        
        # loss
        #self.loss = tf.keras.losses.MSE
        
        
        # layers
        self.dense1 = tf.keras.layers.Dense(400,activation = 'relu',dtype='float32')
        self.dense2 = tf.keras.layers.Dense(300,activation='relu',dtype='float32')
        self.dense3 = tf.keras.layers.Dense(1,dtype='float32') 
        
    #loss
    def loss(self,actual,pred):
        result = tf.keras.losses.MSE(actual,pred)
        return result
    
    def update(self,states,actions,Q_h):
        with tf.GradientTape() as tape:
            Q = self.call(states,actions)
            loss = self.loss(Q,Q_h)

        grad = tape.gradient(loss,self.trainable_variables)
        self.opt.apply_gradients(zip(grad, self.trainable_variables))
        #print('critic loss: ', loss ,"\n" )
        return loss
    
    #predict
    def call(self,states,actions):
        x = tf.concat([states,actions],-1)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
    
    
class SAC(tf.keras.Model):
    def __init__(self,
                 state_space,
                 action_space,
                 capacity = 1000,
                 batch = 1, 
                 tau=0.999,
                 gamma=0.9,
                 actor_lr = 0.001, 
                 critic_lr = 0.0001,
                 variance = 1.0):
        super(SAC,self).__init__()
        # tensorboard callbacks
        self.cb = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=1/3, patience=2, min_lr=1e-4),
                   tf.keras.callbacks.ModelCheckpoint('weights/weights.{epoch:02d}-{val_binary_accuracy:.2f}.cpkt',
                                          monitor='val_binary_accuracy', save_best_only=True, save_weights_only=True),
                   tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3),
                   tf.keras.callbacks.TensorBoard()]
        
        
        #hyperparameters
        self.batch = batch
        self.tau = tau
        self.gamma = gamma
        self.actor_lr = actor_lr
        self.critic_lr = critic_lr
        self.noise_flag = 1
        self.std = np.sqrt(variance)
        
        
        #spaces
        self.action_space = action_space
        self.state_space = state_space
        self.state_action_space = self.action_space + self.state_space
        
        # replay buffer
        self.replay_buffer = ReplayBuffer(self.state_space,self.action_space,capacity,self.batch)
        
        # models
        self.critic = Critic(critic_lr = critic_lr)        
        self.actor = Actor(self.action_space,actor_lr=actor_lr,critic = self.critic)
        #self.critic.compile(optimizer = self.critic.opt,loss = self.critic.loss)
        #self.actor.compile(optimizer = self.actor.opt,loss = self.actor.loss)
        
        # target models
        self.target_actor = Actor(self.action_space,actor_lr=actor_lr,critic = self.critic)
        self.target_critic = Critic(critic_lr = critic_lr)  
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())
        
        #cpc
        #self.cpc = CPC(code_size=128, predict_terms=4, terms=4, units=256, image_size=64, channels=3)
    
    def store_replay(self,state,next_state,action,reward,done):
        self.replay_buffer.add(state,next_state,action,reward,done)
    
    def set_labels(self,states,new_states,actions,rewards):
        mu = self.target_actor(new_states)
        #print(mu,states)
#         stateactions = tf.concat([states,mu],1)
        Q_h = self.target_critic(new_states,mu)
        y = rewards + self.gamma * Q_h
        #y = np.concatenate(self.y,0).astype('float32') #.reshape((self.minibatch_size,1,1,1))
        #print('y: ',self.y)
        y = tf.reshape(y,(self.replay_buffer.batch,1,1,1))
        return y 
    
        
    def discrete_random_noise(self):
        pass
    
    def update_target_weights(self):   
        tgt_critic_weight = self.target_critic.get_weights()
        tgt_actor_weight = self.target_actor.get_weights()
        actor_weight = self.actor.get_weights()
        critic_weight = self.target_actor.get_weights()
        
        for idx,(part_tgt,part_net) in enumerate(zip(tgt_critic_weight,critic_weight)):
            tgt_critic_weight[idx] = self.tau*part_tgt + (1-self.tau)*part_net
        
        for idx,(part_tgt,part_net) in enumerate(zip(tgt_actor_weight,actor_weight)):
            tgt_actor_weight[idx] = self.tau*part_tgt + (1-self.tau)*part_net
            
        self.target_actor.set_weights(tgt_actor_weight)
        self.target_critic.set_weights(tgt_critic_weight)
            
    def save(self,filename):
        pass
    
    def load(self,filename):
        pass
            

In [8]:
class DataHandler:
    def __init__(self, batch_size, terms, predict_terms=1, image_size=64, color=False, rescale=True, aug=True, is_training=True, method='cpc'):
        self.batch_size = batch_size
        self.terms = terms
        self.predict_terms = predict_terms
        self.image_size = image_size
        self.color = color
        self.rescale = rescale
        self.aug = aug
        self.is_training = is_training
        self.method = method
        self.lena = cv2.imread(os.path.join(base_dir,'lena.jpg'))
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        if self.is_training:
            self.x = x_train/255.0
            self.y = y_train
        else:
            self.x = x_test/255.0
            self.y = y_test
        self.idxs = []
        for i in range(10):
            y = y_train if self.is_training else y_test
            self.idxs.append(np.where(y == i)[0])
        self.n_samples = len(self.y)//terms if self.method == 'cpc' else len(self.y)
        self.shape = self.x.shape
        self.n_batches = self.n_samples//batch_size

    def __iter__(self):
        return self

    def __next__(self):
        return self.cpc_batch() if self.method == 'cpc' else self.benchmark_batch()

    def __len__(self):
        return self.n_batches

    def cpc_batch(self):
        img_labels = np.zeros((self.batch_size, self.terms + self.predict_terms))
        sentence_labels = np.ones((self.batch_size, 1)).astype('int32')
        for bi in range(self.batch_size):
            seed = np.random.randint(10)
            sentence = np.arange(seed, seed + self.terms + self.predict_terms) % 10
            if bi < self.batch_size//2:
                num = np.arange(10)
                predicted = sentence[-self.predict_terms:]
                for i, p in enumerate(predicted):
                    predicted[i] = np.random.choice(num[num != p], 1)
                sentence[-self.predict_terms:] = predicted % 10
                sentence_labels[bi, :] = 0
            img_labels[bi, :] = sentence
        images = self.get_samples(img_labels).reshape((self.batch_size, self.terms+self.predict_terms, self.image_size, self.image_size, 3))
        x_images = images[:, :-self.predict_terms, ...]
        y_images = images[:, -self.predict_terms:, ...]
        idx = np.random.choice(self.batch_size, self.batch_size, replace=False)
        return [x_images[idx], y_images[idx]], sentence_labels[idx]

    def get_samples(self, img_labels):
        idx = []
        for label in img_labels.flatten():
            idx.append(np.random.choice(self.idxs[int(label)], 1)[0])
        img_batch = self.x[idx, :, :]
        if self.aug:
            img_batch = self._aug_batch(img_batch)
        return img_batch

    def _aug_batch(self, img_batch):
        if self.image_size != 28:
            resized = []
            for i in range(img_batch.shape[0]):
                resized.append(cv2.resize(img_batch[i], (self.image_size, self.image_size)))
            img_batch = np.stack(resized)
        img_batch = img_batch.reshape((img_batch.shape[0], 1, self.image_size, self.image_size))
        img_batch = np.concatenate([img_batch, img_batch, img_batch], axis=1)

        if self.color:
            img_batch[img_batch >= 0.5] = 1
            img_batch[img_batch < 0.5] = 0
            for i in range(img_batch.shape[0]):
                x_c = np.random.randint(0, self.lena.shape[0] - self.image_size)
                y_c = np.random.randint(0, self.lena.shape[1] - self.image_size)
                img = self.lena[x_c:x_c+self.image_size, y_c:y_c+self.image_size]
                img = np.array(img).transpose((2, 0, 1))/255.0
                for j in range(3):
                    img[j, :, :] = (img[j, :, :] + np.random.uniform(0, 1))/2.0
                img[img_batch[i, :, :, :] == 1] = 1 - img[img_batch[i, :, :, :] == 1]
                img_batch[i, :, :, :] = img

        if self.rescale:
            img_batch = img_batch * 2 - 1
        img_batch = img_batch.transpose((0, 2, 3, 1))
        return img_batch

    def benchmark_batch(self):
        idx = np.random.choice(len(self.x), self.batch_size, replace=False)
        img_batch = self.x[idx]
        label_batch = self.y[idx]
        if self.aug:
            img_batch = self._aug_batch(img_batch)
        label_batch = label_batch.reshape((-1, 1))
        return img_batch, label_batch

In [9]:
# #train loop
# dh_train = DataHandler(64, 4, predict_terms=4, image_size=64, color=True, rescale=True, aug=True, is_training=True, method='cpc')
# dh_test = DataHandler(64, 4, predict_terms=4, image_size=64, color=True, rescale=True, aug=True, is_training=False, method='cpc')
# accuracy_metric_train = tf.keras.metrics.BinaryAccuracy()
# loss_metric_train = tf.keras.metrics.BinaryCrossentropy()
# accuracy_metric_test = tf.keras.metrics.BinaryAccuracy()
# loss_metric_test = tf.keras.metrics.BinaryCrossentropy()
# cpc = CPCModel(code_size=128, predict_terms=4, terms=4, units=256, image_size=64, channels=3)
# optim = tf.keras.optimizers.Adam(1e-3)
# cb = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=1/3, patience=2, min_lr=1e-4),
#       tf.keras.callbacks.ModelCheckpoint('weights/weights.{epoch:02d}-{val_binary_accuracy:.2f}.cpkt',
#                                           monitor='val_binary_accuracy', save_best_only=True, save_weights_only=True),
#       tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3),
#       tf.keras.callbacks.TensorBoard()]
# cpc.compile(optimizer=optim, loss='binary_crossentropy', metrics=['binary_accuracy'])
# cpc.fit(x=dh_train, epochs=10, validation_data=dh_test, steps_per_epoch=60000//64, validation_steps=10000//64, callbacks=cb)


In [10]:
%tensorboard --logdir {logs_base_dir}

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [11]:
# train loop params


episodes = 10
episode_steps = 200
buffer_size = 10000
batch_size = 16

# pybullet setup
env = gym.make('HalfCheetahBulletEnv-v0')
env.render(mode = 'human')
env._max_episode_steps = episode_steps



C:\Users\dtara\anaconda3\envs\csci-7000-rl\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [12]:
writer = tf.summary.create_file_writer(log_dir)


#get spaces
state_space = env.observation_space.shape[0]
action_space = env.action_space.shape[0]
print(state_space,action_space)

26 6


In [13]:
state = env.reset()
sac = SAC(action_space=action_space,state_space=state_space,capacity = buffer_size,batch = batch_size)

#fill replay buffer
env._max_episode_steps = buffer_size
sac.replay_buffer.fill_buffer(buffer_size, state, episode_steps) # self,timesteps,state,prev_timesteps
env._max_episode_steps = episode_steps


env = gym.wrappers.Monitor(env, "cheetah_testing", video_callable=lambda i_episode: True, force="true")
state = env.reset()

for episode in range(episodes):
    sumreward = 0
    for step in range(episode_steps):
        #print(observation)
        print('t: ',step, ' :episode: ',episode)
        #print('state: ',state)
        
        # get action
        state = tf.cast(tf.reshape(state,(1,1,state_space)),dtype='float32')
        #print(state)
        tensor_action = sac.actor(state)+sac.actor.continous_noise()
        action = tensor_action[0][0]
        #print('action: ',action)
        
        #get loss
        q_loss = sac.critic(state,tensor_action)
        
        print('q_loss: ', q_loss[0][0].numpy())
        with writer.as_default():
            tf.summary.scalar('Squared QLosses (qtarget - qval)^2', q_loss[0][0][0].numpy(),
                              step=episode * episode_steps + step + 1)
        
        # execute action
        next_state, reward, done, info = env.step(action)
        sumreward += reward

        # store transitions
        sac.store_replay(state,next_state,action,reward,done)
        
        #print('state: ',state)
        #print('next_state: ',next_state)
        #print('action: ',action)
        #print('reward: ',reward)

        #sample minibatch from data
        states,next_states,actions,rewards,not_done = sac.replay_buffer.sample()
        
        #set labels y_i
        y = sac.set_labels(states,next_states,actions,rewards)
        
        # update critic net
        sac.critic.update(states, actions, y)

        
        
        #losses[episode*timesteps + t] = loss
        #losses[i_episode*timesteps+] = history.history
        
        #update actor net
        sac.actor.update(states,actions)
        #print('weight check: ',rl.actor.get_weights(),'\n')
        
        #update target nets
        sac.update_target_weights()
        
        state = next_state
        if done:
            state = env.reset()
            #rewards[episode] = sumreward
            sac.save('baseline')
            print("Episode {} finished after {} timesteps with reward {}".format(episode,step+1,sumreward))
            break
print('done') 
sac.save('baseline')

sim test:  10000 : 10000
step:  9999
done seeding replay buffer
t:  0  :episode:  0
q_loss:  [-0.06228454]
t:  1  :episode:  0
q_loss:  [0.09520219]
t:  2  :episode:  0
q_loss:  [0.34018242]
t:  3  :episode:  0
q_loss:  [0.54331976]
t:  4  :episode:  0
q_loss:  [0.92615634]
t:  5  :episode:  0
q_loss:  [1.8218089]
t:  6  :episode:  0
q_loss:  [2.3007166]
t:  7  :episode:  0
q_loss:  [2.0020857]
t:  8  :episode:  0
q_loss:  [2.7484913]
t:  9  :episode:  0
q_loss:  [3.3600137]
t:  10  :episode:  0
q_loss:  [3.3977118]
t:  11  :episode:  0
q_loss:  [3.4216654]
t:  12  :episode:  0
q_loss:  [6.0536513]
t:  13  :episode:  0
q_loss:  [6.745388]
t:  14  :episode:  0
q_loss:  [9.2706785]
t:  15  :episode:  0
q_loss:  [10.621215]
t:  16  :episode:  0
q_loss:  [12.045348]
t:  17  :episode:  0
q_loss:  [12.432444]
t:  18  :episode:  0
q_loss:  [11.861871]
t:  19  :episode:  0
q_loss:  [13.798497]
t:  20  :episode:  0
q_loss:  [14.066951]
t:  21  :episode:  0
q_loss:  [18.003277]
t:  22  :episode:

t:  193  :episode:  0
q_loss:  [17522.553]
t:  194  :episode:  0
q_loss:  [19177.742]
t:  195  :episode:  0
q_loss:  [18278.627]
t:  196  :episode:  0
q_loss:  [19521.115]
t:  197  :episode:  0
q_loss:  [19768.455]
t:  198  :episode:  0
q_loss:  [21610.7]
t:  199  :episode:  0
q_loss:  [20634.273]
Episode 0 finished after 200 timesteps with reward -308.05953228154533
t:  0  :episode:  1
q_loss:  [13984.978]
t:  1  :episode:  1
q_loss:  [12786.444]
t:  2  :episode:  1
q_loss:  [8402.745]
t:  3  :episode:  1
q_loss:  [14044.644]
t:  4  :episode:  1
q_loss:  [12010.685]
t:  5  :episode:  1
q_loss:  [9924.485]
t:  6  :episode:  1
q_loss:  [18154.23]
t:  7  :episode:  1
q_loss:  [15755.139]
t:  8  :episode:  1
q_loss:  [16547.217]
t:  9  :episode:  1
q_loss:  [21098.438]
t:  10  :episode:  1
q_loss:  [12657.94]
t:  11  :episode:  1
q_loss:  [17253.943]
t:  12  :episode:  1
q_loss:  [20102.191]
t:  13  :episode:  1
q_loss:  [20294.96]
t:  14  :episode:  1
q_loss:  [20394.176]
t:  15  :episod

t:  187  :episode:  1
q_loss:  [166474.2]
t:  188  :episode:  1
q_loss:  [206180.28]
t:  189  :episode:  1
q_loss:  [181147.1]
t:  190  :episode:  1
q_loss:  [171821.2]
t:  191  :episode:  1
q_loss:  [208020.22]
t:  192  :episode:  1
q_loss:  [206438.97]
t:  193  :episode:  1
q_loss:  [225113.9]
t:  194  :episode:  1
q_loss:  [215332.36]
t:  195  :episode:  1
q_loss:  [215462.81]
t:  196  :episode:  1
q_loss:  [202332.4]
t:  197  :episode:  1
q_loss:  [220357.83]
t:  198  :episode:  1
q_loss:  [197487.89]
t:  199  :episode:  1
q_loss:  [229351.17]
Episode 1 finished after 200 timesteps with reward -330.17027182843157
t:  0  :episode:  2
q_loss:  [145325.72]
t:  1  :episode:  2
q_loss:  [117001.18]
t:  2  :episode:  2
q_loss:  [93106.67]
t:  3  :episode:  2
q_loss:  [132437.75]
t:  4  :episode:  2
q_loss:  [133328.73]
t:  5  :episode:  2
q_loss:  [129661.29]
t:  6  :episode:  2
q_loss:  [171365.86]
t:  7  :episode:  2
q_loss:  [145334.2]
t:  8  :episode:  2
q_loss:  [150358.75]
t:  9  :

t:  182  :episode:  2
q_loss:  [649432.94]
t:  183  :episode:  2
q_loss:  [595867.25]
t:  184  :episode:  2
q_loss:  [600930.44]
t:  185  :episode:  2
q_loss:  [715268.06]
t:  186  :episode:  2
q_loss:  [747910.25]
t:  187  :episode:  2
q_loss:  [734481.06]
t:  188  :episode:  2
q_loss:  [670475.2]
t:  189  :episode:  2
q_loss:  [718773.56]
t:  190  :episode:  2
q_loss:  [691543.7]
t:  191  :episode:  2
q_loss:  [701269.06]
t:  192  :episode:  2
q_loss:  [727371.9]
t:  193  :episode:  2
q_loss:  [640109.75]
t:  194  :episode:  2
q_loss:  [729940.9]
t:  195  :episode:  2
q_loss:  [774793.75]
t:  196  :episode:  2
q_loss:  [730563.06]
t:  197  :episode:  2
q_loss:  [721586.25]
t:  198  :episode:  2
q_loss:  [659932.94]
t:  199  :episode:  2
q_loss:  [606988.2]
Episode 2 finished after 200 timesteps with reward -325.11902252487624
t:  0  :episode:  3
q_loss:  [356805.72]
t:  1  :episode:  3
q_loss:  [521859.53]
t:  2  :episode:  3
q_loss:  [342366.53]
t:  3  :episode:  3
q_loss:  [454365.

t:  175  :episode:  3
q_loss:  [1526315.4]
t:  176  :episode:  3
q_loss:  [1529262.2]
t:  177  :episode:  3
q_loss:  [1459194.6]
t:  178  :episode:  3
q_loss:  [1709610.9]
t:  179  :episode:  3
q_loss:  [1678732.]
t:  180  :episode:  3
q_loss:  [1549901.9]
t:  181  :episode:  3
q_loss:  [1574145.4]
t:  182  :episode:  3
q_loss:  [1424729.6]
t:  183  :episode:  3
q_loss:  [1357887.]
t:  184  :episode:  3
q_loss:  [1598331.6]
t:  185  :episode:  3
q_loss:  [1375341.9]
t:  186  :episode:  3
q_loss:  [1515232.9]
t:  187  :episode:  3
q_loss:  [1579467.5]
t:  188  :episode:  3
q_loss:  [1648171.8]
t:  189  :episode:  3
q_loss:  [1519815.2]
t:  190  :episode:  3
q_loss:  [1690824.1]
t:  191  :episode:  3
q_loss:  [1576495.4]
t:  192  :episode:  3
q_loss:  [1712257.6]
t:  193  :episode:  3
q_loss:  [1618699.6]
t:  194  :episode:  3
q_loss:  [1537045.]
t:  195  :episode:  3
q_loss:  [1472072.6]
t:  196  :episode:  3
q_loss:  [1585837.5]
t:  197  :episode:  3
q_loss:  [1384478.8]
t:  198  :epis

...

t:  164  :episode:  5
q_loss:  [4540270.]
t:  165  :episode:  5
q_loss:  [5340295.]
t:  166  :episode:  5
q_loss:  [6315006.]
t:  167  :episode:  5
q_loss:  [4269986.]
t:  168  :episode:  5
q_loss:  [5721571.]
t:  169  :episode:  5
q_loss:  [4858600.5]
t:  170  :episode:  5
q_loss:  [4833285.5]
t:  171  :episode:  5
q_loss:  [5961397.5]
t:  172  :episode:  5
q_loss:  [5093538.5]
t:  173  :episode:  5
q_loss:  [5184145.5]
t:  174  :episode:  5
q_loss:  [5480113.]
t:  175  :episode:  5
q_loss:  [4780179.]
t:  176  :episode:  5
q_loss:  [4505474.]
t:  177  :episode:  5
q_loss:  [4087709.8]
t:  178  :episode:  5
q_loss:  [5430246.5]
t:  179  :episode:  5
q_loss:  [5124083.5]
t:  180  :episode:  5
q_loss:  [5530363.]
t:  181  :episode:  5
q_loss:  [5094844.]
t:  182  :episode:  5
q_loss:  [4958255.]
t:  183  :episode:  5
q_loss:  [5193737.]
t:  184  :episode:  5
q_loss:  [5393168.5]
t:  185  :episode:  5
q_loss:  [4479499.]
t:  186  :episode:  5
q_loss:  [5481799.]
t:  187  :episode:  5
q_l

t:  158  :episode:  6
q_loss:  [7768181.5]
t:  159  :episode:  6
q_loss:  [7488349.5]
t:  160  :episode:  6
q_loss:  [8553947.]
t:  161  :episode:  6
q_loss:  [9345622.]
t:  162  :episode:  6
q_loss:  [8042291.5]
t:  163  :episode:  6
q_loss:  [8001247.5]
t:  164  :episode:  6
q_loss:  [8056372.]
t:  165  :episode:  6
q_loss:  [8940566.]
t:  166  :episode:  6
q_loss:  [7392304.]
t:  167  :episode:  6
q_loss:  [7968020.5]
t:  168  :episode:  6
q_loss:  [7251179.]
t:  169  :episode:  6
q_loss:  [7453575.]
t:  170  :episode:  6
q_loss:  [7481879.]
t:  171  :episode:  6
q_loss:  [8170308.]
t:  172  :episode:  6
q_loss:  [7585330.5]
t:  173  :episode:  6
q_loss:  [8202967.]
t:  174  :episode:  6
q_loss:  [8876518.]
t:  175  :episode:  6
q_loss:  [9117678.]
t:  176  :episode:  6
q_loss:  [9783241.]
t:  177  :episode:  6
q_loss:  [7430881.]
t:  178  :episode:  6
q_loss:  [8181994.]
t:  179  :episode:  6
q_loss:  [8359511.5]
t:  180  :episode:  6
q_loss:  [7098323.]
t:  181  :episode:  6
q_los

t:  154  :episode:  7
q_loss:  [10689670.]
t:  155  :episode:  7
q_loss:  [9942099.]
t:  156  :episode:  7
q_loss:  [11469766.]
t:  157  :episode:  7
q_loss:  [11414585.]
t:  158  :episode:  7
q_loss:  [10467331.]
t:  159  :episode:  7
q_loss:  [14156492.]
t:  160  :episode:  7
q_loss:  [11113188.]
t:  161  :episode:  7
q_loss:  [8592547.]
t:  162  :episode:  7
q_loss:  [11529096.]
t:  163  :episode:  7
q_loss:  [13164876.]
t:  164  :episode:  7
q_loss:  [11583287.]
t:  165  :episode:  7
q_loss:  [11894413.]
t:  166  :episode:  7
q_loss:  [12150080.]
t:  167  :episode:  7
q_loss:  [11472050.]
t:  168  :episode:  7
q_loss:  [12783398.]
t:  169  :episode:  7
q_loss:  [11340599.]
t:  170  :episode:  7
q_loss:  [11759049.]
t:  171  :episode:  7
q_loss:  [10375804.]
t:  172  :episode:  7
q_loss:  [12404107.]
t:  173  :episode:  7
q_loss:  [12180462.]
t:  174  :episode:  7
q_loss:  [12320259.]
t:  175  :episode:  7
q_loss:  [11910271.]
t:  176  :episode:  7
q_loss:  [11620921.]
t:  177  :epi

q_loss:  [17188728.]
t:  147  :episode:  8
q_loss:  [16293106.]
t:  148  :episode:  8
q_loss:  [16720484.]
t:  149  :episode:  8
q_loss:  [15357780.]
t:  150  :episode:  8
q_loss:  [14413444.]
t:  151  :episode:  8
q_loss:  [16159556.]
t:  152  :episode:  8
q_loss:  [15789732.]
t:  153  :episode:  8
q_loss:  [14894539.]
t:  154  :episode:  8
q_loss:  [12387948.]
t:  155  :episode:  8
q_loss:  [12603402.]
t:  156  :episode:  8
q_loss:  [14798418.]
t:  157  :episode:  8
q_loss:  [15174630.]
t:  158  :episode:  8
q_loss:  [14551860.]
t:  159  :episode:  8
q_loss:  [16482001.]
t:  160  :episode:  8
q_loss:  [12824777.]
t:  161  :episode:  8
q_loss:  [17965858.]
t:  162  :episode:  8
q_loss:  [14735945.]
t:  163  :episode:  8
q_loss:  [17333858.]
t:  164  :episode:  8
q_loss:  [14151151.]
t:  165  :episode:  8
q_loss:  [15140310.]
t:  166  :episode:  8
q_loss:  [15192336.]
t:  167  :episode:  8
q_loss:  [16946496.]
t:  168  :episode:  8
q_loss:  [15577996.]
t:  169  :episode:  8
q_loss:  [1

t:  139  :episode:  9
q_loss:  [20744522.]
t:  140  :episode:  9
q_loss:  [19995374.]
t:  141  :episode:  9
q_loss:  [19298988.]
t:  142  :episode:  9
q_loss:  [19204238.]
t:  143  :episode:  9
q_loss:  [19738216.]
t:  144  :episode:  9
q_loss:  [19005086.]
t:  145  :episode:  9
q_loss:  [23102576.]
t:  146  :episode:  9
q_loss:  [24694982.]
t:  147  :episode:  9
q_loss:  [18702970.]
t:  148  :episode:  9
q_loss:  [21337136.]
t:  149  :episode:  9
q_loss:  [20222514.]
t:  150  :episode:  9
q_loss:  [20381524.]
t:  151  :episode:  9
q_loss:  [20640312.]
t:  152  :episode:  9
q_loss:  [23029644.]
t:  153  :episode:  9
q_loss:  [19228884.]
t:  154  :episode:  9
q_loss:  [21595746.]
t:  155  :episode:  9
q_loss:  [18873676.]
t:  156  :episode:  9
q_loss:  [19070630.]
t:  157  :episode:  9
q_loss:  [19282968.]
t:  158  :episode:  9
q_loss:  [19273560.]
t:  159  :episode:  9
q_loss:  [21225920.]
t:  160  :episode:  9
q_loss:  [23280950.]
t:  161  :episode:  9
q_loss:  [21776490.]
t:  162  :e

 https://datascience.stackexchange.com/questions/13216/intuitive-explanation-of-noise-contrastive-estimation-nce-loss(InfoNCE Loss )
<br>
Representation Learning with Contrastive Predictive Coding
<br>
https://github.com/gdao-research/cpc/blob/master/cpc/data_handler.py (CPC)
<br>
https://github.com/davidtellez/contrastive-predictive-coding/blob/master/train_model.py (CPC)
<br>
https://github.com/MishaLaskin/curl/blob/23b0880708c29b078b0a25e62ff31fb587587b18/utils.py#L123 (replay buffer and SAC)
<br>
https://github.com/marload/DeepRL-TensorFlow2/blob/master/A2C/A2C_Discrete.py (A2C)
<br>
https://github.com/germain-hug/Deep-RL-Keras/blob/master/A3C/a3c.py (A3C)
<br>
https://github.com/tensorflow/agents/blob/v0.5.0/tf_agents/agents/sac/sac_agent.py (SAC)